# Esercizio 2 - Content2Form

In questo esercizio partendo dalle stesse definizioni date dagli studenti usate nell'esercizio 1, si vuole provare a inferire il miglior synset corrispondente a quelle definizioni.  
Si tratta di **Onomasiologic search**.

### Idea:
- Esattamente come nell'esercizio 1, si salva l'input in un dizionario e si pulisce il dizionario
- Si trovano le parole più frequenti per ogni lista di definizioni relative ad ogni termine (come nell'esercizio 1)
- Si considerano le parole più frequenti come **genus** 
- Si preleva da wordnet synset name, definition e examples per *ogni* **iponimo** di ogni **genus**
- Si sceglie il synset con la definizione più simile alla lista delle definizioni, ovvero con più parole in comune

In [55]:
import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import wordnet as wn

### Leggi il file di input

Salvo le varie definizioni degli studenti in un dizionario con:
- chiave: parola (in ordine Emotion, Person, Revenge, Brick)
- valore: lista di definizioni date dagli studenti

In [56]:
df = pd.read_excel('../data/definizioni.xlsx')
students = df.columns[1:].tolist()
words = df[df.columns[0]].tolist()
def_dict = {}
for definition in words:
    def_dict[definition] = []
for student in students:
    for definition in words:
        def_dict[definition].append(df[student][df[df.columns[0]] == definition].values[0])

# def_dict

### Ripulisci il dictionary delle definizioni

Salvo le definizioni pulite in un dizionario strutturato come il precedente

In [57]:
stopwords = nltk.corpus.stopwords.words('english')
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
lemmatizer = nltk.stem.WordNetLemmatizer()

# Tokenize and lemmatize definitions and remove stopwords and punctuation
def clean_definitions(dict):
    cleaned = dict.copy()
    for key in cleaned:
        definitions = [d for d in cleaned[key] if d == d]
        tmp = []
        for definition in definitions:
            tmp.append([lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords])
        cleaned[key] = tmp
    return cleaned

clean_def_dict = clean_definitions(def_dict)

### Trova i genus

Per ogni parola, calcolo le *n* parole più frequenti nelle definizioni date dagli studenti, usando la funzione `most_common` di `Counter`.  
Queste saranno considerate i **genus**.

In [58]:
# Get the most common words in the definitions of a word
def get_common_words(dict, word, n):
    words = []
    for definition in dict[word]:
        words += definition
    return Counter(words).most_common(n)

for word in clean_def_dict:
    print(word, get_common_words(clean_def_dict, word, 5))

Emotion [('feeling', 13), ('human', 8), ('feel', 8), ('something', 7), ('state', 5)]
Person [('human', 29), ('person', 6), ('living', 4), ('individual', 3), ('certain', 3)]
Revenge [('someone', 14), ('anger', 8), ('feeling', 7), ('action', 6), ('emotion', 6)]
Brick [('used', 24), ('object', 16), ('material', 16), ('construction', 16), ('build', 13)]


### Trova tutti gli iponimi di ogni genus

Per ogni genus, trovo tutti gli iponimi e salvo i loro nomi, definizioni e esempi in una lista ripulita.

In [69]:
# Find all hyponyms for all synsets of a word
def find_hyponyms(word):
    synsets = wn.synsets(word)
    hyponyms = []
    for synset in synsets:
        # hyponyms += synset.hyponyms()
        # Closure is used to get all hyponyms for all direct hyponyms
        hyponyms += synset.closure(lambda s:s.hyponyms())
    return set(hyponyms)

# Save hyponyms name, definition and examples in a dictionary
def get_hyponyms_dict(word):
    hyponyms = find_hyponyms(word)
    hyponyms_dict = {}
    for hyponym in hyponyms:
        defs = hyponym.definition()
        examples = hyponym.examples()
        tmp = []
        tmp.append(defs)
        for example in examples:
            tmp.append(example)
        hyponyms_dict[hyponym.name()] = tmp

    return hyponyms_dict

# Preprocess hyponyms definition and examples
def clean_hyponyms_dict(dict):
    cleaned = dict.copy()
    for key in cleaned:
        for definition in cleaned[key]:
            cleaned[key] = [lemmatizer.lemmatize(token.lower()) for token in tokenizer.tokenize(definition) if token.lower() not in stopwords]
    return cleaned

# hypo = get_hyponyms_dict('human')
# cleaned = clean_hyponyms_dict(hypo)
# print(hypo)
# print(cleaned)

### Trova il synset più simile alle definizioni

Per ogni iponimo, calcolo la similarità tra la lista delle definizioni date dagli studenti e la definizione dell'iponimo.  
La similarità è calcolata come il numero di parole in comune tra le due liste.

In [66]:
# Compute the similarity between a definition and a hyponym
def compute_similarity(definition, hyponym):
    definition = set(definition)
    hyponym = set(hyponym)
    return len(definition.intersection(hyponym))